In [1]:
import pandas as pd
import folium
from folium.plugins import HeatMapWithTime
from map import check_cols
import json
from shapely.geometry import Point, Polygon

In [2]:
# Read data files
df = []
for year in range(2017, 2022):
    data = pd.read_csv(f'../../usc_data/sc_loc{year}.csv', low_memory=False)
    data['year'] = year

    # Check if the columns are consistent
    _, data = check_cols('lat', 'lon', data, "")

    df.append(data)

# Combine the dataframes
df = pd.concat(df, ignore_index=True)
len_0 = len(df)  # Initial length
print(f"Length of the dataset: {len_0:,}")

Length of the dataset: 694,335


In [3]:
# Convert lat and lon to correct decimal degrees
df['lat'] = df['lat'] / 1_000_000
df['lon'] = - (df['lon'] / 1000000)  # Note the negative sign for longitude

### Data Filtering for South Carolina

In [4]:
# Load the GeoJSON file
with open("south carolina.geojson", 'r') as f:
    sc_geojson = json.load(f)

# Extract coordinates and create a Shapely polygon
sc_coords = sc_geojson['geometry']['coordinates'][0]
sc_polygon = Polygon(sc_coords)

In [ ]:
# Filter points and calculate exclusion percentage
len_1 = df.shape[0]  # New length

df['in_sc'] = df.apply(lambda row: sc_polygon.contains(Point(row['lon'], row['lat'])), axis=1)
df = df[df['in_sc']]  # Keep only points within South Carolina

print(f"Points after pre-processing : {len_1:,}")
print(f"Points within SC            : {df.shape[0]:,}")
print(f"Excluded points             : {(len_1 - df.shape[0]):,}")
print(f"Exclusion percentage        : {(len_1 - df.shape[0])/ len_1:.2%}")

In [ ]:
# Create a list to store data for each year
data_by_year = []

for year in range(2017, 2022):
    year_data = df[df['year'] == year]
    data_by_year.append(year_data[['lat', 'lon']].values.tolist())

In [ ]:
# Create a map centered on South Carolina
sc_center_lat, sc_center_lon = 33.8361, -81.1637  # Approximate center of SC
m = folium.Map(location=[sc_center_lat, sc_center_lon], zoom_start=7)

In [ ]:
# Add borders to the map for South Carolina
bordersStyle = {
    'color': 'green',
    'weight': 2,
    'fillColor': 'blue',
    'fillOpacity': 0.1
}

# File (`south carolina.geojson`) downloaded from https://github.com/glynnbird/usstatesgeojson/blob/master/south%20carolina.geojson
# File (`South Carolina County Boundaries.geojson`) downloaded from https://cartographyvectors.com/map/1123-south-carolina-with-county-boundaries
folium.GeoJson(
    data=(open("South Carolina County Boundaries.geojson", 'r').read()),
    name="South Carolina",
    style_function=lambda x: bordersStyle).add_to(m);

In [ ]:
# Add the heatmap with time component
HeatMapWithTime(data_by_year,
                name="Heatmap with Time",
                radius=15,
                auto_play=True,
                position='bottomright',
                max_opacity=0.8,
                index=[str(year) for year in range(2017, 2022)]
                ).add_to(m)

# Add a layer control
folium.LayerControl().add_to(m);

In [ ]:
# Save the map
file_name: str = f"../maps/heat_map.html"
m.save(file_name)
print(f"Map saved as {file_name}")